<a href="https://colab.research.google.com/github/KoMurase/giraffe_and_elephant/blob/master/giraffe_or_elephant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_suf8cte
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_suf8cte
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=3b4ddd61d3c975e20f058264d35b87b0c630a0d5982411f8f187ea02f6c5c5e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-sq4g2wxq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
!git clone https://github.com/NVIDIA/cuda-samples/
!cp cuda-samples/Common/* /usr/local/include

fatal: destination path 'cuda-samples' already exists and is not an empty directory.
cp: -r not specified; omitting directory 'cuda-samples/Common/FreeImage'
cp: -r not specified; omitting directory 'cuda-samples/Common/UtilNPP'


In [0]:
import torch 
from  torch import nn,optim
from torch.utils.data import (
Dataset,DataLoader,TensorDataset
)
import tqdm


from torchvision.datasets import ImageFolder 
from torchvision import transforms

train_imgs = ImageFolder("/content/gdrive/My Drive/giraffe_and_elephant/train",
                  transform = transforms.Compose(
                  [
                      transforms.RandomCrop(150),　#画像のサイズに合わせた
                      transforms.ToTensor()
                  ]))

test_imgs = ImageFolder("/content/gdrive/My Drive/giraffe_and_elephant/test",
                  transform = transforms.Compose(
                  [
                      transforms.RandomCrop(150),
                      transforms.ToTensor()
                  ]))

#DataLoaderを作成
train_loader = DataLoader(
    train_imgs,batch_size=32,shuffle=True
)

test_loader = DataLoader(
    test_imgs,batch_size=32,shuffle=False
)

In [0]:
print(train_imgs.classes)

['elephant', 'giraffe']


In [0]:
print(train_imgs.class_to_idx)

{'elephant': 0, 'giraffe': 1}


### **Pytorchで転移学習**

In [0]:
from torchvision import models 

#学習済みresnet18をロード
net = models.resnet18(pretrained=True)

#すべてのパラメータを微分対象外にする
for p in net.parameters():
  p.requires_grad=False
  
#最後の線形層を付け替える
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

モデルの定義は以上

次にモデルの訓練関数を書いていく

In [0]:
def eval_net(net, data_loader, device="cpu"):
  #DropoutやBatchNormを無効化
  net.eval()
  ys = []
  ypreds = []
  
  for x, y in data_loader:
    #toメソッドで計算を実行するデバイスに転送する
    x = x.to(device)
    y = y.to(device)
    #確率が最大となるクラスを予測
    #ここではforward(推論)の計算だけなので自動微分に
    #必要な処理はoffにして余計な計算を省く
    with torch.no_grad():
      _,y_pred = net(x).max(1)
    ys.append(y)
    ypreds.append(y_pred)
  #ミニバッチごとの予測結果を一つにまとめる
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  #予測精度を計算
  acc = (ys == ypreds).float().sum() / len(ys)
  return acc.item()



In [0]:
def train_net(net,train_loader,test_loader,only_fc=True,
             optimizer_cls = optim.Adam,loss_fn=nn.CrossEntropyLoss(),n_iter=10,device='cpu'):
  train_losses = []
  train_acc = []
  val_acc = []
  if only_fc:
    #最後の線形層のパラメータのみを,optimizerに渡す
    optimizer = optimizer_cls(net.fc.parameters())
  else:
    optimizer = optimizer_cls(net.parameters())
  
  for epoch in range(n_iter):
    running_loss = 0.0
    #ネットワークを訓練モードにする
    net.train()
    n = 0
    n_acc = 0
    #tqdmを使用してプログレスバーを出す
    for i ,(xx,yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      h = net(xx)
      loss = loss_fn(h,yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
      _,y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()
    train_losses.append(running_loss / i)
    #訓練データの予測精度
    train_acc.append(n_acc / n)
    #検証データの予測精度
    val_acc.append(eval_net(net, test_loader,device))
    #このepochでの結果を表示
    print(epoch, train_losses[-1],train_acc[-1],val_acc[-1],flush=True)

In [0]:
#ネットワークの全パラメータをGPUに転送
net.to('cuda:0')

#訓練を実行
train_net(net,train_loader,test_loader,n_iter=20,device='cuda:0')





  0%|          | 0/26 [00:00<?, ?it/s]



  4%|▍         | 1/26 [00:00<00:04,  6.11it/s]



  8%|▊         | 2/26 [00:00<00:03,  6.19it/s]



 12%|█▏        | 3/26 [00:00<00:03,  6.28it/s]



 15%|█▌        | 4/26 [00:00<00:03,  6.09it/s]



 19%|█▉        | 5/26 [00:00<00:03,  6.11it/s]



 23%|██▎       | 6/26 [00:00<00:03,  6.24it/s]



 27%|██▋       | 7/26 [00:01<00:03,  6.17it/s]



 31%|███       | 8/26 [00:01<00:02,  6.23it/s]



 35%|███▍      | 9/26 [00:01<00:02,  6.33it/s]



 38%|███▊      | 10/26 [00:01<00:02,  6.39it/s]



 42%|████▏     | 11/26 [00:01<00:02,  6.39it/s]



 46%|████▌     | 12/26 [00:01<00:02,  6.42it/s]



 50%|█████     | 13/26 [00:02<00:01,  6.61it/s]



 54%|█████▍    | 14/26 [00:02<00:01,  6.68it/s]



 58%|█████▊    | 15/26 [00:02<00:01,  6.67it/s]



 62%|██████▏   | 16/26 [00:02<00:01,  6.63it/s]



 65%|██████▌   | 17/26 [00:02<00:01,  6.36it/s]



 69%|██████▉   | 18/26 [00:02<00:01,  6.24it/s]



 73%|███████▎  | 19/26 [00:02<00:01,  6.35it/

0 0.37024151027202606 0.8523573200992556 0.98499995470047






  0%|          | 0/26 [00:00<?, ?it/s]



  4%|▍         | 1/26 [00:00<00:03,  6.68it/s]



  8%|▊         | 2/26 [00:00<00:03,  6.63it/s]



 12%|█▏        | 3/26 [00:00<00:03,  6.68it/s]



 15%|█▌        | 4/26 [00:00<00:03,  6.63it/s]



 19%|█▉        | 5/26 [00:00<00:03,  6.62it/s]



 23%|██▎       | 6/26 [00:00<00:03,  6.62it/s]



 27%|██▋       | 7/26 [00:01<00:02,  6.82it/s]



 31%|███       | 8/26 [00:01<00:02,  7.00it/s]



 35%|███▍      | 9/26 [00:01<00:02,  7.18it/s]



 38%|███▊      | 10/26 [00:01<00:02,  7.17it/s]



 42%|████▏     | 11/26 [00:01<00:02,  7.15it/s]



 46%|████▌     | 12/26 [00:01<00:02,  6.90it/s]



 50%|█████     | 13/26 [00:01<00:01,  6.88it/s]



 54%|█████▍    | 14/26 [00:02<00:01,  6.82it/s]



 58%|█████▊    | 15/26 [00:02<00:01,  6.81it/s]



 62%|██████▏   | 16/26 [00:02<00:01,  6.75it/s]



 65%|██████▌   | 17/26 [00:02<00:01,  6.68it/s]



 69%|██████▉   | 18/26 [00:02<00:01,  6.75it/s]



 73%|███████▎  | 19/26 [00:02<00:01,  6.93it/

1 0.0834086386114359 0.9913151364764268 0.98499995470047






  0%|          | 0/26 [00:00<?, ?it/s]



  4%|▍         | 1/26 [00:00<00:03,  7.65it/s]



  8%|▊         | 2/26 [00:00<00:03,  7.44it/s]



 12%|█▏        | 3/26 [00:00<00:03,  7.31it/s]



 15%|█▌        | 4/26 [00:00<00:03,  7.24it/s]



 19%|█▉        | 5/26 [00:00<00:02,  7.13it/s]



 23%|██▎       | 6/26 [00:00<00:02,  6.97it/s]



 27%|██▋       | 7/26 [00:00<00:02,  6.97it/s]



 31%|███       | 8/26 [00:01<00:02,  7.09it/s]



 35%|███▍      | 9/26 [00:01<00:02,  7.14it/s]



 38%|███▊      | 10/26 [00:01<00:02,  7.01it/s]



 42%|████▏     | 11/26 [00:01<00:02,  6.94it/s]



 46%|████▌     | 12/26 [00:01<00:02,  6.87it/s]



 50%|█████     | 13/26 [00:01<00:01,  6.71it/s]



 54%|█████▍    | 14/26 [00:02<00:01,  6.54it/s]



 58%|█████▊    | 15/26 [00:02<00:01,  6.47it/s]



 62%|██████▏   | 16/26 [00:02<00:01,  6.66it/s]



 65%|██████▌   | 17/26 [00:02<00:01,  6.68it/s]



 69%|██████▉   | 18/26 [00:02<00:01,  6.83it/s]



 73%|███████▎  | 19/26 [00:02<00:01,  6.89it/

...

17 0.010814331583678722 1.0 0.9950000047683716






  0%|          | 0/26 [00:00<?, ?it/s]



  4%|▍         | 1/26 [00:00<00:03,  7.84it/s]



  8%|▊         | 2/26 [00:00<00:03,  7.79it/s]



 12%|█▏        | 3/26 [00:00<00:02,  7.69it/s]



 15%|█▌        | 4/26 [00:00<00:02,  7.70it/s]



 19%|█▉        | 5/26 [00:00<00:02,  7.53it/s]



 23%|██▎       | 6/26 [00:00<00:02,  7.45it/s]



 27%|██▋       | 7/26 [00:00<00:02,  7.27it/s]



 31%|███       | 8/26 [00:01<00:02,  7.17it/s]



 35%|███▍      | 9/26 [00:01<00:02,  7.03it/s]



 38%|███▊      | 10/26 [00:01<00:02,  6.94it/s]



 42%|████▏     | 11/26 [00:01<00:02,  6.76it/s]



 46%|████▌     | 12/26 [00:01<00:02,  6.64it/s]



 50%|█████     | 13/26 [00:01<00:01,  6.54it/s]



 54%|█████▍    | 14/26 [00:02<00:01,  6.49it/s]



 58%|█████▊    | 15/26 [00:02<00:01,  6.46it/s]



 62%|██████▏   | 16/26 [00:02<00:01,  6.54it/s]



 65%|██████▌   | 17/26 [00:02<00:01,  6.73it/s]



 69%|██████▉   | 18/26 [00:02<00:01,  6.87it/s]



 73%|███████▎  | 19/26 [00:02<00:01,  6.81it/

18 0.010088915950618685 1.0 0.9950000047683716






  0%|          | 0/26 [00:00<?, ?it/s]



  4%|▍         | 1/26 [00:00<00:03,  7.34it/s]



  8%|▊         | 2/26 [00:00<00:03,  7.21it/s]



 12%|█▏        | 3/26 [00:00<00:03,  6.99it/s]



 15%|█▌        | 4/26 [00:00<00:03,  6.76it/s]



 19%|█▉        | 5/26 [00:00<00:03,  6.82it/s]



 23%|██▎       | 6/26 [00:00<00:02,  6.98it/s]



 27%|██▋       | 7/26 [00:01<00:02,  7.01it/s]



 31%|███       | 8/26 [00:01<00:02,  6.84it/s]



 35%|███▍      | 9/26 [00:01<00:02,  6.69it/s]



 38%|███▊      | 10/26 [00:01<00:02,  6.59it/s]



 42%|████▏     | 11/26 [00:01<00:02,  6.60it/s]



 46%|████▌     | 12/26 [00:01<00:02,  6.64it/s]



 50%|█████     | 13/26 [00:01<00:01,  6.80it/s]



 54%|█████▍    | 14/26 [00:02<00:01,  6.76it/s]



 58%|█████▊    | 15/26 [00:02<00:01,  6.74it/s]



 62%|██████▏   | 16/26 [00:02<00:01,  6.86it/s]



 65%|██████▌   | 17/26 [00:02<00:01,  7.05it/s]



 69%|██████▉   | 18/26 [00:02<00:01,  7.25it/s]



 73%|███████▎  | 19/26 [00:02<00:00,  7.11it/

19 0.010614412724971772 1.0 0.9950000047683716


In [0]:

class IdentityLayer(nn.Module):
  def forward(self, x):
    return x

net = models.resnet18(pretrained = True)
for p in net.parameters():
  p.requires_grad = False
net.fc = IdentityLayer()



In [0]:

"""#自分で定義する場合
class FlattenLayer(nn.Module):
  def forward(self,x):
  sizes = x.size()
  return x.view(size[0],-1)
conv_net = nn.Sequential(
    nn.Conv2d(3,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64,128,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    FlattenLayer()
    
)
"""
